# 2) Identification of problems in the data with an initial solution (5b)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import scipy.stats as stats
import statsmodels.api as sm
import pylab as py

from numpy import isnan
from sklearn.impute import KNNImputer

In [ ]:
measurements = pd.read_csv("Data Source/007/measurements.csv", sep='\t', lineterminator='\n', na_values='?')
stations = pd.read_csv("Data Source/007/stations.csv", sep='\t', lineterminator='\n')

In [ ]:
# making sure that the data was properly loaded. 
measurements.head(3)

In [ ]:
stations.head(3)

## 2.1) Data Cleaning & Data Preprocessing

In this section we will have a closer look at the quality of the data for each dataset seperatly.
- We will check for dublicated reccords, inconsistent formats, missing values, skewed values and more.  

In [ ]:
measurements.duplicated().sum()

Based on this, we can see that there are 179 dublicated reccords in the measurements data frame. we could potentially just remove all dublicates, but it would be too early to do so at this point, since we dont know the full context of the data frame. we are missing the timestamp for the reccords. we will first have a look at the other file, and then determine a course of action

In [ ]:
stations.duplicated().sum()

unlike the measurements, we do not experience any dublicated reccords in the stations dataset. so at this point, one would assume that the two files should be interconnected.

We therefore start by looking at the amount of reccords, to see if they are equal to eachother.

### 2.1.1) Initial data cleaning

In this section, we will start off by converting the dataset to a format which we believe would be more suitable for the comming tasks, where the changes include:
* correction of inappropriate data structure
* follow up on duplicate records
* inconsistent formats
* missing values
* skewed values.
* and more

#### 2.1.1.1) Stations dataset

When looking at the formatting of the 'revision' column, we can see that there are at lease two types of formatting, dd/mm/yyyy, hh:mm:ss and then also only dd/mm/yyyy. We replace the existing formating with a datetime format. Also, we add two new columns which only comprize of the year and month, this is done for the sake of simplefying some functions when plotting and etc. 

as for the location column, since we dont nessecarily wish to see both the contenent and city on the plots, we split the values into two new columns, namely ['Contenent'] and ['City'], and remove the original column.

In [ ]:
stations.head(1)

In [ ]:
stations['revision'] = pd.to_datetime(stations["revision"]).dt.date
stations = stations.set_index(pd.DatetimeIndex(stations['revision']))

stations['year'] = pd.to_datetime(stations['revision']).dt.year
stations['month'] = pd.to_datetime(stations['revision']).dt.month
stations[['Region', 'City']] = stations['location'].str.split('/', 1, expand=True)
stations = stations.drop(columns=['location'])

# df['QoS'].unique()
stations['QoS'] = stations['QoS'].replace(['accep','maitennce'], ['acceptable','maintenance'])

#### 2.1.1.2) measurements dataset

as it turnes out, the datatype for the warning was initially a floating point, which was used to represent a boolean state. 

In [ ]:
measurements.head(1)

In [ ]:
measurements['warning'] = measurements['warning'].astype(bool)

In [ ]:
measurements.info()

#### 2.1.1.3) KNN Imputation

checking for missing values in the dataset

In [ ]:
measurements.isna().sum()

As we can see, there are 15 columns which have missing values, we can see this by counting the isna() boolean results, and as these are greater than 0 there are values missing.

The KNNImputer is a data transform that is first configured based on the method used to estimate the missing values The default distance measure is a Euclidean distance measure that is NaN aware
The number of neighbors is set to 5 by default and can be configured by the n_neighbors argument. 

in order to read the missing values as NaN, we need to add *na_values='?'* to the *read_csv()* function.

In [ ]:
knnimputer = KNNImputer(n_neighbors=5)
measurements.iloc[:,:] = knnimputer.fit_transform(measurements)

to validate, we can print use the same functions as before like printing out the datqaset to see if it has the same structure and get the sum of the NaN values. 

In [ ]:
measurements.head(3)

In [ ]:
measurements.isna().sum()

Everything looks fine !!

#### 2.1.1.4) Data Normalization and Standardization

In [ ]:
measurements.duplicated().sum()

In [ ]:
print("reccord count of stations: {0}".format(len(stations.index)))
print("reccord count of measurements: {0}".format(len(measurements.index)))

as it turnes out, the two files are no where near the same length. so we need to have a closer look at alternative commonalities of the two files. and as it turnes out, both files consist of the longditude and latitude coordinates.

## Data Preprocessing

In the following we are joining the data series from both files, and the only common denumerator between the two files are the longditude and latitude coordinates. 

In [ ]:
df = pd.merge(stations, measurements, how='inner', left_on = 'longitude', right_on = 'longitude')
df = df.set_index(pd.DatetimeIndex(df['revision']))
df = df.drop_duplicates()
df['latitude'] = df['latitude_y']
df = df.drop(columns=['latitude_x', 'latitude_y', 'revision'])


df['warning'] = df['warning'].astype(bool)

In [ ]:
df.info()

In [ ]:
filepath = 'Dataset.csv'
df.to_csv(filepath)

# ------------------------------------------END------------------------------------------

In [ ]:
rslt_df = df['2014-01-01':'2015-01-01']
sns.displot(rslt_df, x="NOx", bins=20, hue='warning')

In [ ]:
import numpy as np
import statsmodels.api as sm
import pylab as py

sm.qqplot(df['NOx'], line='45')

In [ ]:


rslt_df = df.drop(['latitude', 'longitude', 'month', 'year', 'H2CO', 'PRES', 'C2H3NO5', 'CO', 'PM2.5', 'CH4', 'SO2', 'O3', 'PAHs', 'PM10', 'NH3', 'NOx', 'CFCs', 'Pb'], axis=1)

sns.set_theme(style="ticks")
sns.pairplot(rslt_df, hue="warning")

In [ ]:
df_tmp = df['2014-01-01':'2016-01-12'].drop(['year', 'latitude', 'longitude','month', 'PRES'], axis=1)
df_tmp.plot(figsize=(12,6))

In [ ]:
df_tmp = df.drop(['year', 'latitude', 'longitude','month', 'PRES', 'warning', 'H2CO', 'C2H3NO5', 'CO', 'PM2.5', 'CH4', 'SO2', 'O3', 'PAHs', 'PM10', 'NH3', 'NOx'], axis=1)
df_tmp.plot(kind='hist', bins=50)

In [ ]:
df['TEMP'].plot(kind='hist', bins=50)

In [ ]:
sns.boxplot(x = "year", y = "TEMP", data=df)

In [ ]:
sns.lineplot(x = "year", y = "TEMP", data=df)
plt.show()

In [ ]:
df.duplicated().sum()

**At this point, we need to make a judgement call based on the following.**

    1) what we can see is that we have thousands of measurements, and allthough they might be completely valid, they are out of context as long as we dont have the timestamp on them.
    
    2) we can see that the count of reccords are not impacted by a inner or outer join, so thats fine, but we do experience a discrepancy between the longditude and latitude. 
    - Joining on the longditude column results in 24785 reccords with 353 dublicates and 
    - Joining on the latitude column results in 24737 reccords with 352 dublicated values
    
since joining on the longditude column yeals in a bigger result, we will continue the analasys based on these, and we will also remove the dublicated reccords



In [ ]:
df['QoS'].unique()

In [ ]:
# sns.lineplot(data=df, x=['2016':'2017'], y="TEMP")

In [ ]:
sns.lineplot(x = "year", y = "TEMP", data=df)
plt.show()

In [ ]:
sns.lineplot(x = "year", y = "CFCs", data=df)
plt.show()

In [ ]:
sns.relplot(
    data=df, x="year", y="CFCs",
    col="QoS", hue="warning", style="warning",
    kind="line"
)

In [ ]:
sns.relplot(
    data=df, x="year", y="TEMP",
    col="warning", hue="QoS", style="QoS",
    kind="line"
)

In [ ]:
print(df.loc['2014-01-01':'2015-01-01'])

In [ ]:
sns.set_theme(style="dark")

# Plot each year's time series in its own facet
g = sns.relplot(
    data=df,
    x="month", y="TEMP", col="year", hue="year",
    kind="line", palette="crest", linewidth=4, zorder=5,
    col_wrap=3, height=2, aspect=1.5, legend=False,
)

# Iterate over each subplot to customize further
for year, ax in g.axes_dict.items():

    # Add the title as an annotation within the plot
    ax.text(.8, .85, year, transform=ax.transAxes, fontweight="bold")

    # Plot every year's time series in the background
    sns.lineplot(
        data=df, x="month", y="TEMP", units="year",
        estimator=None, color=".7", linewidth=1, ax=ax,
    )

# Reduce the frequency of the x axis ticks
ax.set_xticks(ax.get_xticks()[::2])

# Tweak the supporting aspects of the plot
g.set_titles("")
g.set_axis_labels("", "TEMP")
g.tight_layout()